In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import unicodedata

In [4]:
#df_1 parecería no tener problemas con sus caracteres, df_2 si. 
df_1 = pd.read_csv('cine_nacional_trabajada.csv', error_bad_lines=False, encoding='utf-8')
df_2 = pd.read_csv('cine_nacional_trabajada_aj_man.csv', encoding= 'latin1', error_bad_lines=False)

Skipping line 3: expected 4 fields, saw 6
Skipping line 15: expected 4 fields, saw 5
Skipping line 22: expected 4 fields, saw 5
Skipping line 43: expected 4 fields, saw 5
Skipping line 58: expected 4 fields, saw 6
Skipping line 113: expected 4 fields, saw 5
Skipping line 118: expected 4 fields, saw 6
Skipping line 279: expected 4 fields, saw 5
Skipping line 330: expected 4 fields, saw 6
Skipping line 402: expected 4 fields, saw 5
Skipping line 472: expected 4 fields, saw 5
Skipping line 478: expected 4 fields, saw 7
Skipping line 518: expected 4 fields, saw 5
Skipping line 559: expected 4 fields, saw 5
Skipping line 599: expected 4 fields, saw 7
Skipping line 639: expected 4 fields, saw 12
Skipping line 680: expected 4 fields, saw 5
Skipping line 681: expected 4 fields, saw 7
Skipping line 727: expected 4 fields, saw 6
Skipping line 743: expected 4 fields, saw 8
Skipping line 770: expected 4 fields, saw 6
Skipping line 833: expected 4 fields, saw 7
Skipping line 846: expected 4 fields,

In [5]:
#Sacamos 18 datos sin sinopsis
df_1.columns = ['ID', 'Titulo','Anio de Produccion', 'Sinopsis']

df_1 = df_1[df_1['Sinopsis'].notnull()]

In [6]:
#Sacamos <br> <i> y todo bracket html que haya quedado en el texto.
import re

def remove_html_signs(n):
    return re.sub(r'\<[^>]*\>','',n)

df_1['Sinopsis'] = df_1['Sinopsis'].apply(remove_html_signs)

In [22]:
#Modificando el tipo de dato de unicode a str en la sinopsis
def from_unicode_to_ascii(n):
    return unicodedata.normalize('NFKD', n).encode('ascii','ignore')

df_1['Sinopsis'] = df_1['Sinopsis'].apply(from_unicode_to_ascii)

In [23]:
#creamos lista de stop-words en español
spanish = {
	"a":               "",
	"acuerdo":         "",
	"adelante":        "",
	"ademas":          "",
	"además":          "",
	"adrede":          "",
	"ahi":             "",
	"ahí":             "",
	"ahora":           "",
	"al":              "",
	"alli":            "",
	"allí":            "",
	"alrededor":       "",
	"antano":          "",
	"antaño":          "",
	"ante":            "",
	"antes":           "",
	"apenas":          "",
	"aproximadamente": "",
	"aquel":           "",
	"aquél":           "",
	"aquella":         "",
	"aquélla":         "",
	"aquellas":        "",
	"aquéllas":        "",
	"aquello":         "",
	"aquellos":        "",
	"aquéllos":        "",
	"aqui":            "",
	"aquí":            "",
	"arribaabajo":     "",
	"asi":             "",
	"así":             "",
	"aun":             "",
	"aún":             "",
	"aunque":          "",
	"bajo":            "",
	"bastante":        "",
	"bien":            "",
	"breve":           "",
	"casi":            "",
	"cerca":           "",
	"claro":           "",
	"como":            "",
	"cómo":            "",
	"con":             "",
	"conmigo":         "",
	"contigo":         "",
	"contra":          "",
	"cual":            "",
	"cuál":            "",
	"cuales":          "",
	"cuáles":          "",
	"cuando":          "",
	"cuándo":          "",
	"cuanta":          "",
	"cuánta":          "",
	"cuantas":         "",
	"cuántas":         "",
	"cuanto":          "",
	"cuánto":          "",
	"cuantos":         "",
	"cuántos":         "",
	"de":              "",
	"debajo":          "",
	"del":             "",
	"delante":         "",
	"demasiado":       "",
	"dentro":          "",
	"deprisa":         "",
	"desde":           "",
	"despacio":        "",
	"despues":         "",
	"después":         "",
	"detras":          "",
	"detrás":          "",
	"dia":             "",
	"día":             "",
	"dias":            "",
	"días":            "",
	"donde":           "",
	"dónde":           "",
	"dos":             "",
	"durante":         "",
	"el":              "",
	"él":              "",
	"ella":            "",
	"ellas":           "",
	"ellos":           "",
	"en":              "",
	"encima":          "",
	"enfrente":        "",
	"enseguida":       "",
	"entre":           "",
	"es":              "",
	"esa":             "",
	"ésa":             "",
	"esas":            "",
	"ésas":            "",
	"ese":             "",
	"ése":             "",
	"eso":             "",
	"esos":            "",
	"ésos":            "",
	"esta":            "",
	"está":            "",
	"ésta":            "",
	"estado":          "",
	"estados":         "",
	"estan":           "",
	"están":           "",
	"estar":           "",
	"estas":           "",
	"éstas":           "",
	"este":            "",
	"éste":            "",
	"esto":            "",
	"estos":           "",
	"éstos":           "",
	"ex":              "",
	"excepto":         "",
	"final":           "",
	"fue":             "",
	"fuera":           "",
	"fueron":          "",
	"g":               "",
	"general":         "",
	"gran":            "",
	"ha":              "",
	"habia":           "",
	"había":           "",
	"habla":           "",
	"hablan":          "",
	"hace":            "",
	"hacia":           "",
	"han":             "",
	"hasta":           "",
	"hay":             "",
	"horas":           "",
	"hoy":             "",
	"i":               "",
	"incluso":         "",
	"informo":         "",
	"informó":         "",
	"junto":           "",
	"la":              "",
	"lado":            "",
	"las":             "",
	"le":              "",
	"lejos":           "",
	"lo":              "",
	"los":             "",
	"luego":           "",
	"mal":             "",
	"mas":             "",
	"más":             "",
	"mayor":           "",
	"me":              "",
	"medio":           "",
	"mejor":           "",
	"menos":           "",
	"menudo":          "",
	"mi":              "",
	"mí":              "",
	"mia":             "",
	"mía":             "",
	"mias":            "",
	"mías":            "",
	"mientras":        "",
	"mio":             "",
	"mío":             "",
	"mios":            "",
	"míos":            "",
	"mis":             "",
	"mismo":           "",
	"mucho":           "",
	"muy":             "",
	"nada":            "",
	"nadie":           "",
	"ninguna":         "",
	"no":              "",
	"nos":             "",
	"nosotras":        "",
	"nosotros":        "",
	"nuestra":         "",
	"nuestras":        "",
	"nuestro":         "",
	"nuestros":        "",
	"nueva":           "",
	"nuevo":           "",
	"nunca":           "",
	"os":              "",
	"otra":            "",
	"otros":           "",
	"pais":            "",
	"paìs":            "",
	"para":            "",
	"parte":           "",
	"pasado":          "",
	"peor":            "",
	"pero":            "",
	"poco":            "",
	"por":             "",
	"porque":          "",
	"pronto":          "",
	"proximo":         "",
	"próximo":         "",
	"puede":           "",
	"qeu":             "",
	"que":             "",
	"qué":             "",
	"quien":           "",
	"quién":           "",
	"quienes":         "",
	"quiénes":         "",
	"quiza":           "",
	"quizá":           "",
	"quizas":          "",
	"quizás":          "",
	"raras":           "",
	"repente":         "",
	"salvo":           "",
	"se":              "",
	"sé":              "",
	"segun":           "",
	"según":           "",
	"ser":             "",
	"sera":            "",
	"será":            "",
	"si":              "",
	"sí":              "",
	"sido":            "",
	"siempre":         "",
	"sin":             "",
	"sobre":           "",
	"solamente":       "",
	"solo":            "",
	"sólo":            "",
	"son":             "",
	"soyos":           "",
	"su":              "",
	"supuesto":        "",
	"sus":             "",
	"suya":            "",
	"suyas":           "",
	"suyo":            "",
	"tal":             "",
	"tambien":         "",
	"también":         "",
	"tampoco":         "",
	"tarde":           "",
	"te":              "",
	"temprano":        "",
	"ti":              "",
	"tiene":           "",
	"todavia":         "",
	"todavía":         "",
	"todo":            "",
	"todos":           "",
	"tras":            "",
	"tu":              "",
	"tú":              "",
	"tus":             "",
	"tuya":            "",
	"tuyas":           "",
	"tuyo":            "",
	"tuyos":           "",
	"un":              "",
	"una":             "",
	"unas":            "",
	"uno":             "",
	"unos":            "",
	"usted":           "",
	"ustedes":         "",
	"veces":           "",
	"vez":             "",
	"vosotras":        "",
	"vosotros":        "",
	"vuestra":         "",
	"vuestras":        "",
	"vuestro":         "",
	"vuestros":        "",
	"ya":              "",
	"yo":              ""}


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

sinopsis = df_1['Sinopsis']

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words= spanish)
tfidf = tfidf_vectorizer.fit_transform(sinopsis)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=spanish)
tf = tf_vectorizer.fit_transform(sinopsis)
tf_feature_names = tf_vectorizer.get_feature_names()

In [25]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 10

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

In [26]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])

no_top_words = 10
print 'Modelo NMF:'
display_topics(nmf, tfidf_feature_names, no_top_words)
print '\nModelo LDA:'
display_topics(lda, tf_feature_names, no_top_words)

Modelo NMF:
Topic 0:
documental ficcion argentina guerra acerca mujeres peron cuatro lucha raul
Topic 1:
anos familia padre vive pueblo madre tres hijo hija viaje
Topic 2:
buenos aires ciudad calles semana pueblo haber viaja rosario literatura
Topic 3:
pelicula cine corto diferentes cuenta accion enrique cortometraje carreras filmada
Topic 4:
historia cuenta narra amor trabajo llega viven amigos personajes tres
Topic 5:
dictadura militar ultima argentina desaparecido 1976 periodo federal esconde protagonistas
Topic 6:
hombre mujer trabajo decide familiar convierte hombres comienza casa trabaja
Topic 7:
joven relacion amor chica conoce novia viejo rompe descubre sociedad
Topic 8:
vida argentina cuenta ficcion carga traves ultimos san aventura cotidiana
Topic 9:
film peron mundo policial salir animados dibujos suarez primer narra

Modelo LDA:
Topic 0:
basado 16mm primer blanco film cuento filmado imagenes hechos sucesos
Topic 1:
buenos aires pelicula vida ciudad familia anos padre argent